In [ ]:
import json
import os

class Coffee:
    def __init__(self, name, price, stock):
        self.name = name
        self.price = price
        self.stock = stock

class Inventory:
    def __init__(self, filename='inventory.json'):
        self.filename = filename
        self.items = []
        self.total_sold = 0
        self.total_income = 0
        self.load()

    def load(self):
        if os.path.exists(self.filename):
            with open(self.filename) as f:
                data = json.load(f)
                self.items = [Coffee(d['name'], d['price'], d['stock']) for d in data.get('menu', [])]
                self.total_sold = data.get('total_sold', 0)
                self.total_income = data.get('total_income', 0)
        else:
            self.items = [
                Coffee("Espresso Mantap Jiwa", 300, 10),
                Coffee("Latte", 350, 10),
                Coffee("Cappuccino bukan cina", 450, 10),
                Coffee("Americano Bikin Melek", 330, 10),
            ]

    def save(self):
        data = {
            'menu': [{'name': c.name, 'price': c.price, 'stock': c.stock} for c in self.items],
            'total_sold': self.total_sold,
            'total_income': self.total_income
        }
        with open(self.filename, 'w') as f:
            json.dump(data, f, indent=2)

    def backup(self, backup_file='backup_inventory.json'):
        self.save()
        with open(self.filename) as src, open(backup_file, 'w') as dst:
            dst.write(src.read())
        print("Backup selesai, data aman Bang.")

class Order:
    def __init__(self, inventory):
        self.items = []
        self.inventory = inventory

    def add(self, idx):
        coffee = self.inventory.items[idx]
        if coffee.stock > 0:
            coffee.stock -= 1
            self.items.append(coffee)
            print(f"Oke sip, {coffee.name} sudah dimasukin ke keranjang. Mau nambah lagi?")
        else:
            print(f"Aduh maaf Bang, stok {coffee.name} habis.")

    def total(self):
        return sum(c.price for c in self.items)

    def show(self):
        if not self.items:
            print("Belum ada pesanan nih Bang.")
            return
        print("\nNih pesanan Abang:")
        for i, c in enumerate(self.items, 1):
            print(f"{i}. {c.name} - Rp{c.price}")
        print(f"Total sementara: Rp{self.total()}\n")

    def checkout(self):
        if not self.items:
            print("Lah, keranjang kosong Bang.")
            return
        self.show()
        confirm = input("Bener nih mau langsung bayar? (iya/nggak): ").strip().lower()
        if confirm == 'iya':
            total_items = len(self.items)
            total_price = self.total()
            self.inventory.total_sold += total_items
            self.inventory.total_income += total_price
            self.inventory.save()
            print(f"Order diterima! Makasih banyak Bang. Total yang harus dibayar: Rp{total_price}")
            self.items.clear()
        else:
            print("Yaudah Bang, kapan-kapan aja.")

def report(inventory):
    print("\n--- Laporan Penjualan ---")
    print(f"Total item terjual: {inventory.total_sold}")
    print(f"Total pemasukan: Rp{inventory.total_income}")
    print("------------------------\n")

def main():
    inv = Inventory()
    order = Order(inv)

    while True:
        print("\n--- Menu Kopi Kita ---")
        for i, c in enumerate(inv.items, 1):
            print(f"{i}. {c.name} - Rp{c.price} (Stok: {c.stock})")
        menu = len(inv.items)
        print(f"{menu+1}. Lihat keranjang pesanan")
        print(f"{menu+2}. Checkout")
        print(f"{menu+3}. Laporan penjualan")
        print(f"{menu+4}. Backup data")
        print(f"{menu+5}. Keluar")

        pilih = input("Pilih nomor menu: ").strip()
        if pilih.isdigit():
            pilih = int(pilih)
            if 1 <= pilih <= menu:
                order.add(pilih - 1)
            elif pilih == menu+1:
                order.show()
            elif pilih == menu+2:
                order.checkout()
            elif pilih == menu+3:
                report(inv)
            elif pilih == menu+4:
                inv.backup()
            elif pilih == menu+5:
                print("Makasih banyak udah mampir Bang! Sampai jumpa lagi.")
                break
            else:
                print("Nomor menu nggak ada Bang, coba lagi.")
        else:
            print("Masukin nomor aja Bang, jangan huruf.")

if __name__ == "__main__":
    main()


--- Menu Kopi Kita ---
1. Espresso Mantap Jiwa - Rp300 (Stok: 10)
2. Latte - Rp350 (Stok: 10)
3. Cappuccino bukan cina - Rp450 (Stok: 10)
4. Americano Bikin Melek - Rp330 (Stok: 10)
5. Lihat keranjang pesanan
6. Checkout
7. Laporan penjualan
8. Backup data
9. Keluar
